<div style="padding:20px 30px 30px; 
            color:#004346;
            font-size:40px;
            display:fill;
            text-align:center;
            border-radius:20px;
            border: 5px double;
            border-color:#201E20;
            background-color: #E8F1F2;
            overflow:hidden;
            font-weight:400"> 
<p style="font-weight: bold; text-align: center;">Определение уровня сложности фильмов</p>


</div>

<div style="padding:0px 40px 30px; 
            color:#004346;
            font-size:110%;
            display:fill;
            border-radius:20px;
            border: 5px double;
            border-color:#201E20;
            background-color: #E8F1F2;
            overflow:hidden;
            font-weight:450;"> 
    
__Постановка проблемы:__ Просмотр фильмов на оригинальном языке - это популярный и действенный метод прокачаться при изучении иностранных языков. Важно выбрать фильм, который подходит студенту по уровню сложности, т.е. студент понимал 50-70 % диалогов. Чтобы выполнить это условие, преподаватель должен посмотреть фильм и решить, какому уровню он соответствует. Однако это требует больших временных затрат.
    
__Цель:__ Разработать ML решение для автоматического определения уровня сложности англоязычных фильмов, разработать для неё веб-интерфейс и создать микросервис. 
    
__Описание данных:__

- субтитры фильмов, сохраненные в директориях, названия которых, соответствуют уровню сложности по шкале CEFR([Common European Framework of Reference]('https://ru.wikipedia.org/wiki/%D0%9E%D0%B1%D1%89%D0%B5%D0%B5%D0%B2%D1%80%D0%BE%D0%BF%D0%B5%D0%B9%D1%81%D0%BA%D0%B8%D0%B5_%D0%BA%D0%BE%D0%BC%D0%BF%D0%B5%D1%82%D0%B5%D0%BD%D1%86%D0%B8%D0%B8_%D0%B2%D0%BB%D0%B0%D0%B4%D0%B5%D0%BD%D0%B8%D1%8F_%D0%B8%D0%BD%D0%BE%D1%81%D1%82%D1%80%D0%B0%D0%BD%D0%BD%D1%8B%D0%BC_%D1%8F%D0%B7%D1%8B%D0%BA%D0%BE%D0%BC' 'wikipedia'))
    
- субтитры фильмов и [фaйл xlsx](https://i.postimg.cc/8zXN4BJ6/2023-06-08-11-58-46.png), содержаший название фильмов и уровню сложности по шкале CEFR
    
- список слов, по уровлю сложности Oxford level.
</div>        

<div style="padding:0px 20px 10px; 
            color:#004346;
            font-size:15px;
            display:fill;
            text-align:center;
            border-radius:20px;
            border: 5px double;
            border-color:#201E20;
            background-color: #E8F1F2;
            overflow:hidden;
            font-weight:400"> 

# Используемые библиотеки

</div>

In [1]:
from pathlib import Path
import pysrt
import re
import csv
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import joblib
import random
import optuna
import warnings

from nltk.stem.porter import PorterStemmer
porter_stemmer = PorterStemmer()

from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack

from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer, mean_absolute_error

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV

from catboost import Pool, CatBoostRegressor

nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
# константы
RANDOM_SEED = 42
EARLY_STOPPING_ROUND = 100

HTML = r'<.*?>'
TAG = r'{.*?}'
COMMENTS = r'[\(\[][A-Z ]+[\)\]]'
LETTERS = r'[^a-zA-Z\'.,!? ]'
SPACES = r'([ ])\1+'
DOTS = r'[\.]+'
PUNCTUATION = re.compile('[^а-яa-z\s]')

# настройки блокнота
warnings.filterwarnings('ignore')
pd.options.display.max_colwidth = None
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/denismuhanov/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/denismuhanov/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/denismuhanov/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


<div style="padding:0px 20px 10px; 
            color:#004346;
            font-size:15px;
            display:fill;
            text-align:center;
            border-radius:20px;
            border: 5px double;
            border-color:#201E20;
            background-color: #E8F1F2;
            overflow:hidden;
            font-weight:400"> 

# Загрузка и первичная обработка данных

</div>

In [2]:
try:
    # путь к папке с файлами субтитров
    subtitles_folder = Path('data/subtitles/subtitles_cerf/')
    subtitles_folder2 = 'data/subtitles/subtitles_no_labels'
    excel_file = 'data/subtitles/movies_labels.xlsx'

    # загрузка данных для рассчета статистической информации:
    a1_list = next(csv.reader(open('data/a1.csv', 'r')))
    a2_list = next(csv.reader(open('data/a2.csv', 'r')))
    b1_list = next(csv.reader(open('data/b1.csv', 'r')))
    b2_list = next(csv.reader(open('data/b2.csv', 'r')))
    c1_list = next(csv.reader(open('data/c1.csv', 'r')))
    display('Данные загруженны коректно')
except:
    display('Данные не доступны')

'Данные загруженны коректно'

In [3]:
# функция для первичной обработки текста
def clean_subs(txt):
    txt = re.sub(HTML, ' ', txt) #html тэги меняем на пробел
    txt = re.sub(TAG, ' ', txt) #тэги меняем на пробел
    txt = re.sub(COMMENTS, ' ', txt) #комменты меняем на пробел
    txt = re.sub(LETTERS, ' ', txt) #все что не буквы меняем на пробел
    txt = re.sub(SPACES, r'\1', txt) #повторяющиеся пробелы меняем на один пробел
    txt = re.sub(DOTS, r'.', txt) #многоточие меняем на точку
    txt = txt.encode('ascii', 'ignore').decode() #удаляем все что не ascii символы
    txt = ".".join(txt.lower().split('.')[1:-1]) #удаляем первый и последний субтитр (обычно это реклама)
    txt = txt.replace('. .', '. ')
    return txt

__Получение первой части размеченных данных__

In [4]:
# получение списока файлов субтитров в папке
subtitles_files = subtitles_folder.rglob('*.srt')
# cоздание пустого датафрейма для хранения данных
df = pd.DataFrame(columns=['movie', 'subtitles', 'label'])
# загрузка и первичня обработка субтитров
data = []
for file_path in subtitles_files:
    subs = pysrt.open(str(file_path), encoding='latin-1')
    if len(subs) == 0:
        subs = pysrt.open(str(file_path), encoding='utf-16') #учитываем альтернативную кодировку
    txt = ' '.join([sub.text for sub in subs])
    subtitle_text = clean_subs(txt)
    data.append({'movie': file_path.name[:-4], 'subtitles': subtitle_text, 'label': file_path.parent.name})
    
# объединяем все записи в датасете с помощью функции concat
df = pd.concat([df, pd.DataFrame(data)], ignore_index=True)

__Получение второй части размеченных данных__

In [5]:
# загрузка датафрейма из файла excel
df2 = pd.read_excel(excel_file)
# удаление столбца, не использующего в дальнейшем анализе: `id`
df2 = df2.drop('id', axis=1)
# переименование признаков для последующей конкатинации датасетов
df2.rename(columns = {
    'Movie':'movie',
    'Level':'label'
    }, inplace = True )

In [6]:
# функция для добавления судтитров, по названию фильмов и первичной обработки
def add_srt(x):
    try:
        file_path = subtitles_folder2+x+'.srt'
        subs = pysrt.open(file_path, encoding='latin-1')
        if len(subs) == 0:
            subs = pysrt.open(str(file_path), encoding='utf-16') #учитываем альтернативную кодировку
        txt = ' '.join([sub.text for sub in subs])
        clean_text = clean_subs(txt)
        return clean_text
    except FileNotFoundError:
        return np.nan

In [7]:
# добавление судтитров
df2['subtitles'] = df2['movie'].apply(add_srt)

__Объединие данных__

In [8]:
df = pd.concat([df, df2], ignore_index=True)

<div style="padding: 30px 25px; border: 2px #6495ed solid">


- Субтитры загружены и объединены в датасет
- Проведена первичная обработка субтитров:
    - html тэги заменены на пробел
    - комментарии заменены на пробел
    - все что не является буквами заменены на пробел
    - повторяющиеся пробелы заменены на один пробел
    - многоточия заменены на точку
    - удалены все что не ascii символы
    - удалены первый и последний субтитр (обычно это реклама)

    

    
</div>

<div style="padding:0px 20px 10px; 
            color:#004346;
            font-size:15px;
            display:fill;
            text-align:center;
            border-radius:20px;
            border: 5px double;
            border-color:#201E20;
            background-color: #E8F1F2;
            overflow:hidden;
            font-weight:400"> 

# Предобработка и исследовательский анализ данных

## Общая информация

</div>

In [9]:
display(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 404 entries, 0 to 403
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   movie      404 non-null    object
 1   subtitles  273 non-null    object
 2   label      404 non-null    object
dtypes: object(3)
memory usage: 9.6+ KB


None

- Названия фильмов не несут информацию и сложности диалогов в нем, поэтому: данный признак можно удалить.
- В данных присутствуют пропуски, это связано с тем, что в файле excel, были перечислеыы часть фильмов, распрелеленных по отдельным папкам, названия которых, соответствуют уровню сложности языка. Следует удалить подобные записи.

In [10]:
# удаление признака `movie`
df = df.drop('movie', axis=1)
# удаление записей с пропусками
df = df.dropna()

__Проверим наличие полных дубликатов в данных:__

In [11]:
print(f'Количество полных дубликатов: {df.duplicated().sum()}')

Количество полных дубликатов: 2


- Надичие дубликарованных строк, связано с наличием одинаковых субтитров в двух изначальных наборах данных, их следует удалить.

In [12]:
# удаление дубликатов
df = df.drop_duplicates()

<div style="padding:0px 20px 10px; 
            color:#004346;
            font-size:15px;
            display:fill;
            text-align:center;
            border-radius:20px;
            border: 5px double;
            border-color:#201E20;
            background-color: #E8F1F2;
            overflow:hidden;
            font-weight:400"> 


## Баланс целевого признака

</div>

In [13]:
# вывод количества записей по класскам
#df['label'] = df['label'].apply(lambda x: str(x))
display(df['label'].value_counts())

B2            136
B1             52
C1             39
A2/A2+         25
B1, B2          8
A2              6
A2/A2+, B1      5
Name: label, dtype: int64

- Как видно из сводной информации, существует дисбаланс класов
- Записей по межклассовым (A2/A2+, B1/B2 и тд) не достаточно для обучения классификатора. Небходимо:
    - используя [таблицу CEFR/IELS](https://i.postimg.cc/W4YQxZkJ/2023-06-08-15-23-32.png) преобразобать значения в числовой вариант и в последующем решать задачу регрессии
    
    

In [14]:
cefr_dict = {'A2': 3.25, #среднее значение крайних значений [3.0:3.5]
             'B1': 4.5,  #среднее значение крайних значений [4.0:5.0]
             'B2': 6.0, #среднее значение крайних значений [5.5:6.5]
             'C1': 7.5, #среднее значение крайних значений [7.0:8.0]
             'A2/A2+': 3.5, #верхняя граница
             'A2+': 3.5, #верхняя граница
             'B1, B2': 5.25, #среднее значение верхнего B1 и низнего B2 [5.0:5.5]
             'A2/A2+, B1': 3.75} #среднее значение верхнего A2 и нижнего B1 [3.5:4.0]

In [15]:
df['ielts_index'] = df['label'].apply(lambda x: cefr_dict[x]).copy()

<div style="padding:0px 20px 10px; 
            color:#004346;
            font-size:15px;
            display:fill;
            text-align:center;
            border-radius:20px;
            border: 5px double;
            border-color:#201E20;
            background-color: #E8F1F2;
            overflow:hidden;
            font-weight:400"> 


## Рассчет коофициентов удобочитаемости Флеша-Кинкейда

</div>

Тесты на удобочитаемость [Флеша-Кинкейда](https://en.wikipedia.org/wiki/Flesch%E2%80%93Kincaid_readability_tests#Flesch%E2%80%93Kincaid_grade_level) — это тесты на удобочитаемость , предназначенные для определения того, насколько труден для понимания отрывок на английском языке . Есть два теста: Flesch Reading-Ease и Flesch-Kincaid Grade Level. Хотя они используют одни и те же основные меры (длина слова и длина предложения), они имеют разные весовые коэффициенты.

- Flesch Reading-Ease:
$$
FRE = 206.835 - (words/sentences)*1.015 - (syllables/words)*84.6
$$

- Flesch-Kincaid Grade Level:
$$
FKGL = (words/sentences)*0.39 + (syllables/words)*11.8 - 15.59
$$
где:
- words - количество слов в тексте;
- sentences - количество предложений в тексте;
- syllables - количество слогов в тексте

In [16]:
# функции для рассчета коофициентов удобочитаемости:
def statistics(txt):
    total_sentences = len(re.split(r"[.!?]", txt))
    total_words = len(txt.split(' '))
    total_syllables = sum(txt.count(g) for g in 'aeoiu') + txt.count('y')/2
    return total_sentences, total_words, total_syllables
    
def flesch_reading_ease(txt):
    sentences, words, syllables = statistics(txt)
    fres = 206.835 - (words/sentences)*1.015 - (syllables/words)*84.6
    return fres
    
def flesch_kincaid_grade_level(txt):
    sentences, words, syllables = statistics(txt)
    fkgl = (words/sentences)*0.39 + (syllables/words)*11.8 - 15.59
    return fkgl  

In [17]:
df['fres'] = df['subtitles'].apply(flesch_reading_ease)

In [18]:
df['fkgl'] = df['subtitles'].apply(flesch_kincaid_grade_level)

<div style="padding:0px 20px 10px; 
            color:#004346;
            font-size:15px;
            display:fill;
            text-align:center;
            border-radius:20px;
            border: 5px double;
            border-color:#201E20;
            background-color: #E8F1F2;
            overflow:hidden;
            font-weight:400"> 


## Токенизация и лематизация текста

</div>

- Перед токенизацией текса, необходимо удалить оставшиеся знаки пунктуации и стоп-слова - список слов, которые не влияют на сложность текста, но могут уменьшить точность модели.
- Текенизация - это процесс разделения предложений на слова-компоненты.
- Лемматизация и стемминг текста. Обычно тексты содержат разные грамматические формы одного и того же слова, а также могут встречаться однокоренные слова. Лемматизация и стемминг преследуют цель привести все встречающиеся словоформы к одной, нормальной словарной форме.

In [19]:
# удаление знаков пунктуации
df['subtitles'] = df['subtitles'].apply(lambda x: PUNCTUATION.sub('', x))

___________
__stopwords/word_tokenize__

In [20]:
# функция для удаления стоп-слов и токенизации текста
def stopwords_tokenize(x):
    tokens = word_tokenize(x)
    tokenization = [word for word in tokens if not word in stopwords.words('english')]
    return tokenization

In [21]:
df['subtitles'] = df['subtitles'].apply(stopwords_tokenize)

_________
__stemmer/lemmatizer__

In [22]:
# функция для стеминга и лемматизации
def stemmer_lemmatizer(x):
    stemmer = [porter_stemmer.stem(s) for s in x]
    lemmatizer = [wordnet_lemmatizer.lemmatize(w) for w in stemmer]
    return lemmatizer

In [23]:
df['subtitles'] = df['subtitles'].apply(stemmer_lemmatizer)

<div style="padding:0px 20px 10px; 
            color:#004346;
            font-size:15px;
            display:fill;
            text-align:center;
            border-radius:20px;
            border: 5px double;
            border-color:#201E20;
            background-color: #E8F1F2;
            overflow:hidden;
            font-weight:400"> 


## Рассчет долей слов относительно индекса CEFR

</div>

- Рассчитаем долю слов в тексте, соответствующих списку слов, разделенные по уровням сложности CEFR

In [24]:
# cоздание пустого датафрейма для хранения данных
df_info = pd.DataFrame(columns=['a1', 'a2', 'b1', 'b2', 'c1'])

In [25]:
data_info = []
# функция рассчета доли слов по уровням сложности
def oxford_cefr(x):
    a1 = sum(1 for i in x if i in a1_list)
    a2 = sum(1 for i in x if i in a2_list)
    b1 = sum(1 for i in x if i in b1_list)
    b2 = sum(1 for i in x if i in b2_list)
    c1 = sum(1 for i in x if i in c1_list)
    count_word = a1+a2+b1+b2+c1
    a1 = a1/count_word
    a2 = a2/count_word
    b1 = b1/count_word
    b2 = b2/count_word
    c1 = c1/count_word
    data_info.append({'a1':a1, 'a2':a2, 'b1':b1, 'b2': b2, 'c1':c1})

In [26]:
df['subtitles'].apply(oxford_cefr)

0      None
1      None
2      None
3      None
4      None
       ... 
270    None
271    None
272    None
273    None
274    None
Name: subtitles, Length: 271, dtype: object

In [27]:
# объединяем все записи в датасете с помощью функции concat
df_info = pd.concat([df_info, pd.DataFrame(data_info)], ignore_index=True)
df = pd.concat([df, df_info], axis=1).copy()
# удаление записей с пропусками
df = df.dropna()

<div style="padding: 30px 25px; border: 2px #6495ed solid">


- Изучена общая информация.
- Обработаны дубликаты и пропуски в данных.
- Выявлен дисбаланс классов.
- Удалены стоп-слова.
- Проведена токенизация и лематизация текста.
- Рассчитаны дополнительные признаки:
    - Flesch Reading-Ease
    - Flesch-Kincaid Grade Level
    - Доли слов относительно индекса CEFR
    
</div>

<div style="padding:0px 20px 10px; 
            color:#004346;
            font-size:15px;
            display:fill;
            text-align:center;
            border-radius:20px;
            border: 5px double;
            border-color:#201E20;
            background-color: #E8F1F2;
            overflow:hidden;
            font-weight:400"> 


# Разработка модели ML

- Для обучения моделей машинного обучения по прежнему не хватает записей, но так как субтитры в большенстве своем достаточно длинные, мы в праве разделить их по определенному количеству слов, тем самым увеличив количество записей. При этом рассчитанные ранее статистические данные следует оставить без изменений, тк они были рассчитаны на всем тексте и обладают большей точностью, по сравнению с теми, которые мы можем рассчитать на части данных.
   - выбырем количество: 100 слов 
- Некоторые записи могут содержать меньшее число слов, их следует исключить.

In [28]:
# определение количества слов в субтитрах
df['len'] = df['subtitles'].apply(lambda x: len(x))

In [29]:
df = df[df['len']>=100]

In [30]:
# cоздание пустого датафрейма для хранения данных
df_div = pd.DataFrame(columns=['subtitles','a1', 'a2', 'b1', 'b2', 'c1', 'fres', 'fkgl','label', 'ielts_index'])

In [31]:
# функция для разделения субтитров
len_div = 100 #количество слов в подвыборке, для увеличения наблюдений
data = []
def text_division(x):
    for i in range(len(x['subtitles'])//len_div):
        data.append({'subtitles': ' '.join(x['subtitles'][i*len_div:(i+1)*len_div+1]), 
                     'a1': x['a1'],
                     'a2': x['a2'],
                     'b1': x['b1'],
                     'b2': x['b2'],
                     'c1': x['c1'],
                     'fres': x['fres'],
                     'fkgl': x['fkgl'],
                     'label': x['label'],
                     'ielts_index': x['ielts_index']})               

In [32]:
df.apply(text_division, axis=1)

0      None
1      None
2      None
3      None
4      None
       ... 
265    None
266    None
267    None
268    None
270    None
Length: 263, dtype: object

In [33]:
# jбъединяем все записи в датасете с помощью функции concat
df = pd.concat([df_div, pd.DataFrame(data)], ignore_index=True)

__Выделение обучающей и тестовой выборок__

In [34]:
# признаки для обучения
X = ['subtitles', 'a1', 'a2', 'b1', 'b2', 'c1', 'fres', 'fkgl']
# целевой признак
y = ['ielts_index']

X_train, X_test, y_train, y_test = train_test_split(df[X], df[y], test_size=0.2, random_state=RANDOM_SEED)
print(f'Размер выборок: {X_train.shape};{X_test.shape};{y_train.shape};{y_test.shape}')

Размер выборок: (7826, 8);(1957, 8);(7826, 1);(1957, 1)


<div style="padding: 30px 25px; border: 2px #6495ed solid">

- Увеличен размер выборки за счет разделения субтитров на несколько записей.
- Датафрейм разделен на обучающую и тестовую выборки.
    
</div>

<div style="padding:0px 20px 10px; 
            color:#004346;
            font-size:15px;
            display:fill;
            text-align:center;
            border-radius:20px;
            border: 5px double;
            border-color:#201E20;
            background-color: #E8F1F2;
            overflow:hidden;
            font-weight:400"> 


## RandomForest

- Для обучения модели, необходима векторизация текстовой информации. Поскольку текстовые данные представлены в виде последовательности слов или символов, их необходимо преобразовать в числовой формат, чтобы модель могла работать с ними. Одним из популярных методов векторизации текста является TF-IDF (Term Frequency-Inverse Document Frequency). Этот метод присваивает каждому слову в тексте числовое значение, основанное на его частоте встречаемости в документе (Term Frequency) и обратной частоте встречаемости в корпусе документов (Inverse Document Frequency). Результатом векторизации текста с использованием TF-IDF является числовое представление текста, где каждое слово представлено числовым значением, отражающим его важность в контексте данного текста. Так же необходимо ограничить размерность матрицы, тк в случае сохранения всех параметров, потребуется много времени для реализации данного алгоритма обучения.

__TF-IDF векторизация__

In [41]:
# Векторизация столбца 'subtitles' с помощью TF-IDF
vectorizer = TfidfVectorizer(max_features=25)

X_train_vectorized = vectorizer.fit_transform(X_train['subtitles'])
X_test_vectorized = vectorizer.transform(X_test['subtitles'])
# Вывод размерности матрицы TF-IDF
print("Размерность обучающей матрицы TF-IDF:", X_train_vectorized.shape)
print("Размерность тестовой матрицы TF-IDF:", X_test_vectorized.shape)

# Преобразование векторизованного столбца 'subtitles' в массив NumPy
X_train_vectorized = X_train_vectorized.toarray()
X_test_vectorized = X_test_vectorized.toarray()

# Объединение данных в один набор
X_train_combined = np.hstack((X_train_vectorized, X_train.drop('subtitles', axis=1)))
X_test_combined = np.hstack((X_test_vectorized, X_test.drop('subtitles', axis=1)))

Размерность обучающей матрицы TF-IDF: (7826, 25)
Размерность тестовой матрицы TF-IDF: (1957, 25)


__Обучение модели__

In [42]:
# гиперпараметры модели
param_grid = {
    'n_estimators': [100, 150],
    'max_depth': [5, 7],
    'min_samples_split': [2, 3]
}
# параметры модели
model_RF = RandomForestRegressor(random_state=RANDOM_SEED)
# объект метрики MAE
scorer = make_scorer(mean_absolute_error)
# параметры RandomizedSearchCV
random_search_RF = RandomizedSearchCV(estimator=model_RF,
                                      param_distributions=param_grid,
                                      n_iter=100,
                                      cv=3,
                                      verbose=True,
                                      random_state=RANDOM_SEED,
                                      scoring=scorer,
                                      n_jobs=-1)

In [43]:
# обучение модели
random_search_RF.fit(X_train_combined, y_train.values.ravel())
# сохраним лучшую модель
best_model_RF = random_search_RF.best_estimator_
# сохраним лучшее значение метрики
final_metrics_RF = random_search_RF.best_score_
# вывод результатов
print(f'Оптимальные гиперпараметры:\n{random_search_RF.best_params_}\n{final_metrics_RF}')

Fitting 3 folds for each of 8 candidates, totalling 24 fits
Оптимальные гиперпараметры:
{'n_estimators': 150, 'min_samples_split': 2, 'max_depth': 5}
0.5313494403967689


<div style="padding: 30px 25px; border: 2px #6495ed solid">

- Оптимальные гитерпараметры для алгоритма RandomForest:
    - 'n_estimators': 150
    - 'min_samples_split': 2
    - 'max_depth': 5
    
- Метрика качества MAE на обучающих данных при кроссвалидации составляет: 0.53
    
</div>

<div style="padding:0px 20px 10px; 
            color:#004346;
            font-size:15px;
            display:fill;
            text-align:center;
            border-radius:20px;
            border: 5px double;
            border-color:#201E20;
            background-color: #E8F1F2;
            overflow:hidden;
            font-weight:400"> 


## CatBoost

- CatBoost - это градиентный бустинговый алгоритм, разработанный компанией Yandex. Он является мощным инструментом для задач классификации и регрессии, который обладает рядом преимуществ и особенностей.

- Одной из основных преимуществ CatBoost является его способность работать напрямую с категориальными признаками, включая текстовые данные. В отличие от многих других алгоритмов, CatBoost может обрабатывать категориальные признаки без необходимости их предварительной векторизации.

In [122]:
# тестовые признаки 
text_features = ['subtitles']
# pool
train_pool = Pool(X_train,
                  label=y_train,
                  text_features=text_features)

test_pool = Pool(X_test,
                 text_features=text_features)
# целевой признак тестовой выборки
y_test_cb = y_test

__optuna-подбор оптимальных гиперпараметров__

In [117]:
# выборки для подбора гиперпараметров CatBoost
X_train, X_valid, y_train, y_valid = train_test_split(df[X], df[y], test_size=0.2, random_state=RANDOM_SEED)
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.1, random_state=RANDOM_SEED)
# размер выборок
X_train.shape, y_train.shape, X_valid.shape, y_valid.shape, X_test.shape, y_test.shape

((7043, 8), (7043, 1), (1957, 8), (1957, 1), (783, 8), (783, 1))

In [51]:
# гиперпараметры optuna
def objective(trial):
    param = {}
    param['learning_rate'] = trial.suggest_discrete_uniform("learning_rate", 0.01, 0.02, 0.001)
    param['depth'] = trial.suggest_int('depth', 9, 15)
    param['l2_leaf_reg'] = trial.suggest_discrete_uniform('l2_leaf_reg', 1.0, 5.5, 0.5)
    param['min_child_samples'] = trial.suggest_categorical('min_child_samples', [1, 4, 8, 16, 32])
    param['grow_policy'] = 'Depthwise'
    param['iterations'] = 1000
    param['use_best_model'] = True
    param['eval_metric'] = 'MAE'
    param['od_type'] = 'iter'
    param['od_wait'] = 20
    param['random_state'] = RANDOM_SEED
    param['logging_level'] = 'Silent'
    param['text_features'] = text_features
    
    regressor = CatBoostRegressor(**param)

    regressor.fit(X_train.copy(), y_train.copy(),
                  eval_set=[(X_test.copy(), y_test.copy())],
                  early_stopping_rounds=EARLY_STOPPING_ROUND)
    loss = mean_absolute_error(y_valid, regressor.predict(X_valid.copy()))
    return loss

In [54]:
%%time
study = optuna.create_study(study_name=f'catboost-seed{RANDOM_SEED}')
study.optimize(objective, n_trials=1000, n_jobs=-1, timeout=2400)

[I 2023-06-03 19:07:55,451] A new study created in memory with name: catboost-seed42
/var/folders/1l/cj5j7tnj6q347f0pj242vpfm0000gn/T/ipykernel_3885/3340873305.py:3: FutureWarning: suggest_discrete_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., step=...) instead.
  param['learning_rate'] = trial.suggest_discrete_uniform("learning_rate", 0.01, 0.02, 0.001)
/var/folders/1l/cj5j7tnj6q347f0pj242vpfm0000gn/T/ipykernel_3885/3340873305.py:5: FutureWarning: suggest_discrete_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., step=...) instead.
  param['l2_leaf_reg'] = trial.suggest_discrete_uniform('l2_leaf_reg', 1.0, 5.5, 0.5)
[I 2023-06-03 19:13:16,118] Trial 5 finished with value: 0.11235527533125683 and parameters: {'learning_rate': 0.02, 'depth': 10, 'l2_leaf_reg': 4.5, '

[I 2023-06-03 19:26:51,243] Trial 15 finished with value: 0.10743209239881818 and parameters: {'learning_rate': 0.019000000000000003, 'depth': 11, 'l2_leaf_reg': 5.0, 'min_child_samples': 32}. Best is trial 2 with value: 0.09038012162176244.
/var/folders/1l/cj5j7tnj6q347f0pj242vpfm0000gn/T/ipykernel_3885/3340873305.py:3: FutureWarning: suggest_discrete_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., step=...) instead.
  param['learning_rate'] = trial.suggest_discrete_uniform("learning_rate", 0.01, 0.02, 0.001)
/var/folders/1l/cj5j7tnj6q347f0pj242vpfm0000gn/T/ipykernel_3885/3340873305.py:5: FutureWarning: suggest_discrete_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., step=...) instead.
  param['l2_leaf_reg'] = trial.suggest_discrete_uniform('l2_leaf_reg', 1.0, 5.5,

[I 2023-06-03 19:40:25,698] Trial 19 finished with value: 0.11227949550735124 and parameters: {'learning_rate': 0.011, 'depth': 13, 'l2_leaf_reg': 2.5, 'min_child_samples': 16}. Best is trial 2 with value: 0.09038012162176244.
/var/folders/1l/cj5j7tnj6q347f0pj242vpfm0000gn/T/ipykernel_3885/3340873305.py:3: FutureWarning: suggest_discrete_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., step=...) instead.
  param['learning_rate'] = trial.suggest_discrete_uniform("learning_rate", 0.01, 0.02, 0.001)
/var/folders/1l/cj5j7tnj6q347f0pj242vpfm0000gn/T/ipykernel_3885/3340873305.py:5: FutureWarning: suggest_discrete_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., step=...) instead.
  param['l2_leaf_reg'] = trial.suggest_discrete_uniform('l2_leaf_reg', 1.0, 5.5, 0.5)
[I 2023-0

CPU times: user 11h 54min 56s, sys: 7min 26s, total: 12h 2min 23s
Wall time: 1h 14min 44s


In [55]:
# лучшая метрика
study.best_value

0.08805955872892557

In [56]:
# оптимальные гиперпараметры
study.best_params

{'learning_rate': 0.016,
 'depth': 15,
 'l2_leaf_reg': 2.5,
 'min_child_samples': 1}

<div style="padding: 30px 25px; border: 2px #6495ed solid">

- Оптимальные гитерпараметры для алгоритма CatBoost, при использовании алгоритма optuna:
    - 'learning_rate': 0.016
    - 'depth': 15
    - 'l2_leaf_reg': 2.5
    - 'min_child_samples': 1
    
- Метрика качества MAE на обучающих данных при кроссвалидации составляет: 0.09
    ____________
    
- Для дальнейшего обучения выбираем алгоритм CatBoost, показывающий лучшую метрику качества. Однако, чать параметром не будет использоваться далее, что увеличит скорость обучения и предсказаний. В дальнейшем, при необходимости более точных данных, можно их можно будет использовать в модели.
    
</div>

__Обучение модели__

In [118]:
# гиперпараметры модели
parameters = {'verbose': 100,
              'text_features': ['subtitles'],
              'eval_metric': 'MAE',
              'iterations': 1000,
              'learning_rate': 0.2,
              'random_seed':RANDOM_SEED,
              'early_stopping_rounds': 30
             }
# параметры модели
regressor = CatBoostRegressor(**parameters)

In [119]:
# обучение модели
regressor.fit(train_pool)

0:	learn: 0.9101034	total: 95.9ms	remaining: 1m 35s
100:	learn: 0.1426646	total: 3.78s	remaining: 33.7s
200:	learn: 0.0779433	total: 7.45s	remaining: 29.6s
300:	learn: 0.0562830	total: 11.2s	remaining: 26.1s
400:	learn: 0.0441051	total: 15s	remaining: 22.3s
500:	learn: 0.0359515	total: 18.6s	remaining: 18.5s
600:	learn: 0.0305177	total: 22.3s	remaining: 14.8s
700:	learn: 0.0263480	total: 26s	remaining: 11.1s
800:	learn: 0.0228443	total: 29.7s	remaining: 7.37s
900:	learn: 0.0200240	total: 33.4s	remaining: 3.67s
999:	learn: 0.0177868	total: 37s	remaining: 0us


<div style="padding:0px 20px 10px; 
            color:#004346;
            font-size:15px;
            display:fill;
            text-align:center;
            border-radius:20px;
            border: 5px double;
            border-color:#201E20;
            background-color: #E8F1F2;
            overflow:hidden;
            font-weight:400"> 


## Проверка модели на тестовой выборке

In [123]:
# предказание данных
predict = regressor.predict(test_pool)
mae = mean_absolute_error(y_test_cb, predict)
# вывод результатов
print("Средняя абсолютная ошибка (MAE):", mae)

Средняя абсолютная ошибка (MAE): 0.05040942850434126


_________

In [93]:
# сохранение итоговой модели
regressor.save_model('catboost_model')

<div style="padding:0px 20px 10px; 
            color:#004346;
            font-size:15px;
            display:fill;
            text-align:center;
            border-radius:20px;
            border: 5px double;
            border-color:#201E20;
            background-color: #E8F1F2;
            overflow:hidden;
            font-weight:400"> 


# Вывод

- В ходе проведения исследования была выполнена предобработка данных, включающая очистку и лемматизацию текстовых признаков. Для векторизации текста был применен метод TF-IDF, который позволяет представить тексты в виде числовых признаков, учитывающих важность каждого термина в документе.

- Для настройки гиперпараметров модели CatBoost был использован фреймворк Optuna. Optuna провел исследование пространства гиперпараметров, оценивая производительность модели с различными комбинациями параметров. Целевая метрика, в данном случае MAE, была определена для оптимизации. В результате подбора гиперпараметров были получены оптимальные значения, которые позволяют достичь наилучшей производительности модели CatBoost на данной задаче и составиляют 0.05 на тестовой выборке.

- Полученная модель будет использоваться в разработке сервиса, предназначенного для определения уровня сложности английского языка в фильмах. Этот сервис будет доступен через платформу Streamlit, что позволит пользователям оценивать и анализировать сложность субтитров и основываться на предсказанных значениях индекса IELTS или CEFR